In [1]:
import pandas as pd
import re
import json
from concurrent.futures import ThreadPoolExecutor
import time

# 开始计时
start_time = time.time()

# 加载数据
with open('A_share_list.json', 'r', encoding='utf-8') as file:
    a_share_list = json.load(file)

df = pd.read_excel('News_original.xlsx')
df['NewsContent'].fillna('', inplace=True)

# 创建公司名称列表及fullname到name的映射
name = []
fullname_to_name = {}
for item in a_share_list:
    name.append(item['name'])
    name.append(item['fullname'])
    fullname_to_name[item['fullname']] = item['name']

# 编译正则表达式
pattern = re.compile('|'.join(map(re.escape, name)), flags=re.IGNORECASE)

def process_batch(start, end):
    temp_results = []
    batch_df = df.iloc[start:end]
    for _, row in batch_df.iterrows():
        matches = pattern.findall(row['NewsContent'])
        if matches:
            # 确保name优先于fullname
            filtered_matches = set()
            for match in matches:
                if match in fullname_to_name:
                    filtered_matches.add(fullname_to_name[match])
                else:
                    filtered_matches.add(match)
            temp_results.append({'NewsID': row['NewsID'], 'NewsContent': row['NewsContent'], 'Explicit_Company': ', '.join(filtered_matches)})
    return temp_results

# 并行处理
executor = ThreadPoolExecutor(max_workers=32)
futures = []
batch_size = 10000

for start in range(0, len(df), batch_size):
    end = start + batch_size
    futures.append(executor.submit(process_batch, start, end))

# 收集结果
results = []
for future in futures:
    results.extend(future.result())

# 转换为 DataFrame
result_df = pd.DataFrame(results)

# 显示部分结果以验证
print(result_df.head())

# 总体运行时间
total_run_time = time.time()
print("Total run time: {:.2f} seconds".format(total_run_time - start_time))

   NewsID                                        NewsContent Explicit_Company
0       1  　　本报记者 田雨 李京华    　　中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...             建设银行
1       2  　　中国农业银行信用卡中心由北京搬到上海了！  　　农行行长杨明生日前在信用卡中心揭牌仪式上...             农业银行
2       3  　　在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段...       外运发展, 中国国航
3       4  　　胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿...             胜利股份
4       8  　　由于全球最大的俄罗斯Uralkaly钾矿被淹，产量大减，同时满洲里口岸铁路在修复线，导致...             冠农股份
Total run time: 21915.75 seconds


In [2]:
result_df

,NewsID,NewsContent,Explicit_Company
0,1,本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...,建设银行
1,2,中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信用卡中心揭牌仪式上...,农业银行
2,3,在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段...,"外运发展, 中国国航"
3,4,胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿...,胜利股份
4,8,由于全球最大的俄罗斯Uralkaly钾矿被淹，产量大减，同时满洲里口岸铁路在修复线，导致...,冠农股份
...,...,...,...
467421,1037031,每经AI快讯，有投资者在投资者互动平台提问：请问公司目前有没有电解槽产能，规划情况能否详细介...,亿华通
467422,1037032,依米康（SZ 300249，收盘价：10.38元）发布公告称，2023年10月12日，依米康...,"中泰证券, 依米康"
467423,1037033,天风证券10月13日发布研报称，给予中核科技（000777.SZ，最新价：13.03元）买入...,"天风证券, 中核科技"
467424,1037034,有投资者提问：抗癌药CPT获批后，公司是否应该按照股权协议继续收购沙东股权，适应症为MM的C...,海特生物


In [3]:
# 保存修改后的 DataFrame 到新的 XLSX 文件
result_df.to_excel('result_df.xlsx', index=False)

In [ ]:
result_df = pd.read_excel('result_df.xlsx')

In [5]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# 开始计时
start_time = time.time()

# 加载预训练的BERT模型和分词器
model_name = "hw2942/bert-base-chinese-finetuning-financial-news-sentiment-v2"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# 检查CUDA可用性，并将模型移至GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    inputs = inputs.to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
        # 获取消极和积极的概率
        negative_prob = probabilities[:, 0].item()
        positive_prob = probabilities[:, 2].item()
        # 如果积极的概率大于消极的概率，返回1，否则返回0
        return int(positive_prob > negative_prob)
    
# 在result_df中进行情感分析
def add_sentiment_labels(df):
    df['label'] = df['NewsContent'].apply(sentiment_analysis)
    return df

# 应用情感分析
result_df_with_sentiment = add_sentiment_labels(result_df.copy())

# 显示部分结果以验证
print(result_df_with_sentiment.head())

# 总体运行时间
total_run_time = time.time()
print("Total run time: {:.2f} seconds".format(total_run_time - start_time))

KeyboardInterrupt: 

In [ ]:
# 保存修改后的 DataFrame 到新的 XLSX 文件
result_df_with_sentiment.to_excel('result_df_with_sentiment.xlsx', index=False)